In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device} device')

using cpu device


In [4]:
# 클래스 정의하기
## 신경망 모델을 nn.Module의 하위클래스로 정의하고 __init__에서 신경망 계층들을 초기화함
## forward 메소드에 입력 데이터에 대한 연산들을 구현함


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
# NeuralNetwork의 인스턴스를 생성하고 이를 device로 이동시킨 뒤 구조 출력하기

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# 모델 사용 위해 입력 데이터 전달 - 일부 백그라운드 연산들과 함께 모델의 forward를 실행하게 됨
# model.forward()를 직접 호출해서는 안됨
# 모델에 입력을 넣어 호출하면 각 class에 대한 raw 예측값이 있는 10차원의 텐서가 반환됨, 이를 nn.Softmax 모듈 인스턴스에 통과시켜 예측 확률 얻을 수 있음

X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted Class : {y_pred}')

Predicted Class : tensor([8])


In [7]:
# 각각의 모델 계층 살펴보기

input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
# Flatten 레이어를 초기화하여 28*28 2D이미지를 784 픽셀값을 갖는 연속된 배열로 반환합니다.
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
# Linear 계층은 저장된 weigt과 bias를 사용하여 입력에 linear trasnformation을 적용하는 모듈

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# ReLU 계층은 입력과 출력 사이에 복잡한 관계를 만들어 냄
## 비선형 활성화 : 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 도움

print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU : {hidden1}')

Before ReLU : tensor([[ 0.0054, -0.8662, -0.5101, -0.1615,  0.2329,  0.0710, -0.2884,  0.1770,
          0.0707,  0.6733,  0.2663,  0.3544, -0.3589, -0.3568,  0.3350, -0.1779,
          0.3456,  0.0441, -0.0232,  0.1232],
        [ 0.2487, -1.1259, -0.4552,  0.2732,  0.3317,  0.0613, -0.0941,  0.1861,
         -0.1735,  0.9402, -0.2034, -0.0833, -0.3405, -0.5075,  0.4993, -0.0342,
          0.3201,  0.4867,  0.0808,  0.2674],
        [ 0.1422, -1.0758, -0.5911,  0.0733,  0.0588,  0.2367,  0.1300,  0.1350,
         -0.2161,  0.6208, -0.2088, -0.1420, -0.4745, -0.5145,  0.4360, -0.2027,
          0.0956,  0.0710, -0.1936, -0.0484]], grad_fn=<AddmmBackward>)


After ReLU : tensor([[0.0054, 0.0000, 0.0000, 0.0000, 0.2329, 0.0710, 0.0000, 0.1770, 0.0707,
         0.6733, 0.2663, 0.3544, 0.0000, 0.0000, 0.3350, 0.0000, 0.3456, 0.0441,
         0.0000, 0.1232],
        [0.2487, 0.0000, 0.0000, 0.2732, 0.3317, 0.0613, 0.0000, 0.1861, 0.0000,
         0.9402, 0.0000, 0.0000, 0.0000, 0.0000, 0.4

In [12]:
# Sequential : 순서를 갖는 모듈의 컨테이너

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10))

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [13]:
# nn.Softmax
# 신경망의 마지막 계층은 [-infinity, infinity] 범위의 raw value인 logits를 반환함.
# logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례 조정됨
# dim 매개변수는 값의 합이 1이되는 차원을 의미함

softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

In [15]:
# 모델 매개변수
## 신경망 내부의 많은 layer는 매개변수화됨 -> 학습 중에 최적화되는 가중치, 편향과 연관되어짐
## nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters(), named_parameters() 메소드로 모든 매개변수에 접근 가능
print('Model Structure :', model, '\n\n')

for name, param in model.named_parameters():
    print(f'Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n')

Model Structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0124,  0.0039, -0.0136,  ..., -0.0202, -0.0210, -0.0224],
        [ 0.0335, -0.0060,  0.0330,  ..., -0.0158,  0.0082, -0.0132]],
       grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0133,  0.0216], grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[-0.0098,  0.0404,  0.0347,  ...,  0.0151,  0.0220,  0.0040],
        [ 0.0205,  0.0420,  0.0197,  ...,  0.0108, -0.0013,  0.0336]],
       grad_fn=<SliceBackward>) 

Layer : linear_relu_stack.2.b